In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Importovanje baze i vizelizacija iste


In [23]:
df = pd.read_csv('iot_devices_train.csv')
df1 = pd.read_csv('iot_devices_test.csv')

In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 298 entries, ack to device_category
dtypes: float64(80), int64(217), object(1)
memory usage: 2.3+ MB


In [24]:
df.head()

,ack,ack_A,ack_B,bytes,bytes_A,bytes_A_B_ratio,bytes_B,ds_field_A,ds_field_B,duration,http_GET,http_POST,http_bytes_avg,http_bytes_entropy,http_bytes_firstQ,http_bytes_max,http_bytes_median,http_bytes_min,http_bytes_stdev,http_bytes_sum,http_bytes_thirdQ,http_bytes_var,http_cookie_count,http_cookie_values_avg,http_cookie_values_entropy,http_cookie_values_firstQ,http_cookie_values_max,http_cookie_values_median,http_cookie_values_min,http_cookie_values_stdev,http_cookie_values_sum,http_cookie_values_thirdQ,http_cookie_values_var,http_count_host,http_count_req_content_type,http_count_resp_code,http_count_resp_content_type,http_count_transactions,http_count_user_agents,http_dom_host_alexaRank,...,B_port_is_54975,B_port_is_80,B_port_is_8080,B_port_is_8280,B_port_is_9543,B_port_is_else,subdomain_is_99sets,subdomain_is_ccc,subdomain_is_else,subdomain_is_feeds,subdomain_is_h10141.www1,subdomain_is_img,subdomain_is_unresolved,subdomain_is_whp.aus1.cold.extweb,subdomain_is_whp.hou9.cold.extweb,subdomain_is_www,subdomain_is_www.cloud,domain_is_dlink,domain_is_else,domain_is_epicurious,domain_is_google,domain_is_hp,domain_is_hpeprint,domain_is_livecdn,domain_is_mako,domain_is_proteussensor,domain_is_samsung,domain_is_unresolved,suffix_is_biz,suffix_is_cloudfront.net,suffix_is_co.il,suffix_is_com,suffix_is_com.sg,suffix_is_else,suffix_is_empty_char_value,suffix_is_googleapis.com,suffix_is_net,suffix_is_org,suffix_is_unresolved,device_category
0,9,5,5,1213,743,0.713924,668,0,0,1.5756,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,security_camera
1,9,5,5,1213,743,1.806874,668,0,0,0.6890,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,security_camera
2,9,5,5,1213,743,0.103124,668,0,0,0.9852,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,security_camera
3,9,5,5,1213,743,1.806874,668,0,0,1.5756,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,security_camera
4,9,5,5,1213,743,1.806874,668,0,0,1.5756,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2147483647,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,security_camera


In [25]:
print(df["device_category"].unique())

['security_camera' 'TV' 'smoke_detector' 'thermostat' 'water_sensor'
 'watch' 'baby_monitor' 'motion_sensor' 'lights' 'socket']


In [26]:
df.isnull().sum()

ack                         0
ack_A                       0
ack_B                       0
bytes                       0
bytes_A                     0
                           ..
suffix_is_googleapis.com    0
suffix_is_net               0
suffix_is_org               0
suffix_is_unresolved        0
device_category             0
Length: 298, dtype: int64

Preprocessing

In [27]:
from sklearn.preprocessing import LabelEncoder
encodings_dictionary = dict()
for c in df.columns:
    if df[c].dtype == "object":
        encodings_dictionary[c] = LabelEncoder()
        df[c] = encodings_dictionary[c].fit_transform(df[c])

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df.drop('device_category',axis=1)
y= df['device_category']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

'''scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)'''

'scaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_test = scaler.transform(X_test)'

Trying out models


In [29]:
from sklearn.svm import SVC

svm=SVC()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)

In [32]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.60      0.94      0.73        16
           1       0.87      0.87      0.87        23
           2       0.28      1.00      0.43        16
           3       1.00      1.00      1.00        19
           4       1.00      0.93      0.97        15
           5       1.00      0.95      0.97        20
           6       0.00      0.00      0.00        25
           7       1.00      0.88      0.93        16
           8       0.78      0.69      0.73        26
           9       1.00      0.21      0.34        24

    accuracy                           0.70       200
   macro avg       0.75      0.75      0.70       200
weighted avg       0.74      0.70      0.67       200

[[15  0  0  0  0  0  0  0  1  0]
 [ 2 20  0  0  0  0  0  0  1  0]
 [ 0  0 16  0  0  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  1  0  0 14  0  0  0  0  0]
 [ 1  0  0  0  0 19  0  0  0  0]
 [ 0  0 25  0  0  0  0  0  0  0]
 [ 1  0  0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C':[0.5,1,10,100,1000], 'gamma':[0.1,0.01,0.001,0.0001,0.00001]}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=True)

grid.fit(X_train,y_train)
grid_predictions = grid.predict(X_test)
print(classification_report(y_test,grid_predictions))
print(confusion_matrix(y_test,grid_predictions))

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: ignored

In [34]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr_predictions = lr.predict(X_test)
print(classification_report(y_test,lr_predictions))
print(confusion_matrix(y_test,lr_predictions))

              precision    recall  f1-score   support

           0       0.78      0.88      0.82        16
           1       0.92      0.96      0.94        23
           2       0.33      1.00      0.50        16
           3       0.95      1.00      0.97        19
           4       0.88      0.93      0.90        15
           5       1.00      0.95      0.97        20
           6       0.38      0.12      0.18        25
           7       0.94      0.94      0.94        16
           8       0.95      0.77      0.85        26
           9       0.90      0.38      0.53        24

    accuracy                           0.76       200
   macro avg       0.80      0.79      0.76       200
weighted avg       0.80      0.76      0.74       200

[[14  0  0  0  0  0  1  1  0  0]
 [ 0 22  0  0  1  0  0  0  0  0]
 [ 0  0 16  0  0  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  1  0  0 14  0  0  0  0  0]
 [ 0  0  0  1  0 19  0  0  0  0]
 [ 0  0 21  0  0  0  3  0  0  1]
 [ 0  0  0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [36]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
print(classification_report(y_test,knn_predictions))
print(confusion_matrix(y_test,knn_predictions))

              precision    recall  f1-score   support

           0       0.70      0.88      0.78        16
           1       0.95      0.87      0.91        23
           2       0.33      0.44      0.38        16
           3       0.86      1.00      0.93        19
           4       1.00      0.93      0.97        15
           5       1.00      0.95      0.97        20
           6       0.45      0.52      0.48        25
           7       0.83      0.94      0.88        16
           8       1.00      0.77      0.87        26
           9       0.50      0.33      0.40        24

    accuracy                           0.74       200
   macro avg       0.76      0.76      0.76       200
weighted avg       0.76      0.74      0.75       200

[[14  0  0  1  0  0  0  0  0  1]
 [ 0 20  0  0  0  0  0  3  0  0]
 [ 0  0  7  0  0  0  5  0  0  4]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  1  0  0 14  0  0  0  0  0]
 [ 0  0  0  1  0 19  0  0  0  0]
 [ 0  0  9  0  0  0 13  0  0  3]
 [ 0  0  0

In [37]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
m = RFECV(RandomForestClassifier(), scoring='accuracy')
m.fit(X_train, y_train)
m_predictions = m.predict(X_test)
print(classification_report(y_test,m_predictions))
print(confusion_matrix(y_test,m_predictions))
#S'obzirom na broj kolona, izuzetno neefikasno

KeyboardInterrupt: ignored

In [39]:
rf = RandomForestClassifier(n_estimators=50)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
print(classification_report(y_test,rf_predictions))
print(confusion_matrix(y_test,rf_predictions))


              precision    recall  f1-score   support

           0       0.88      0.94      0.91        16
           1       1.00      1.00      1.00        23
           2       0.39      0.44      0.41        16
           3       0.90      1.00      0.95        19
           4       1.00      1.00      1.00        15
           5       1.00      0.95      0.97        20
           6       0.45      0.52      0.48        25
           7       1.00      0.88      0.93        16
           8       0.96      0.96      0.96        26
           9       0.56      0.42      0.48        24

    accuracy                           0.80       200
   macro avg       0.81      0.81      0.81       200
weighted avg       0.81      0.80      0.80       200

[[15  0  1  0  0  0  0  0  0  0]
 [ 0 23  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  6  0  0  3]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  1  0 19  0  0  0  0]
 [ 0  0  7  0  0  0 13  0  0  5]
 [ 1  0  0

PCA i isprobavanje modela nad smanjenom bazom

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
pca.n_components_

10

In [ ]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [40]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
print(classification_report(y_test,rf_predictions))
print(confusion_matrix(y_test,rf_predictions))

              precision    recall  f1-score   support

           0       0.79      0.94      0.86        16
           1       1.00      0.96      0.98        23
           2       0.44      0.50      0.47        16
           3       0.95      1.00      0.97        19
           4       1.00      1.00      1.00        15
           5       1.00      0.95      0.97        20
           6       0.58      0.60      0.59        25
           7       1.00      0.88      0.93        16
           8       0.96      0.92      0.94        26
           9       0.55      0.50      0.52        24

    accuracy                           0.81       200
   macro avg       0.83      0.82      0.82       200
weighted avg       0.82      0.81      0.82       200

[[15  0  1  0  0  0  0  0  0  0]
 [ 1 22  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  3  0  0  5]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  1  0 19  0  0  0  0]
 [ 0  0  5  0  0  0 15  0  0  5]
 [ 1  0  0

In [41]:
from sklearn.svm import SVC

svm=SVC()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

           0       0.60      0.94      0.73        16
           1       0.87      0.87      0.87        23
           2       0.28      1.00      0.43        16
           3       1.00      1.00      1.00        19
           4       1.00      0.93      0.97        15
           5       1.00      0.95      0.97        20
           6       0.00      0.00      0.00        25
           7       1.00      0.88      0.93        16
           8       0.78      0.69      0.73        26
           9       1.00      0.21      0.34        24

    accuracy                           0.70       200
   macro avg       0.75      0.75      0.70       200
weighted avg       0.74      0.70      0.67       200

[[15  0  0  0  0  0  0  0  1  0]
 [ 2 20  0  0  0  0  0  0  1  0]
 [ 0  0 16  0  0  0  0  0  0  0]
 [ 0  0  0 19  0  0  0  0  0  0]
 [ 0  1  0  0 14  0  0  0  0  0]
 [ 1  0  0  0  0 19  0  0  0  0]
 [ 0  0 25  0  0  0  0  0  0  0]
 [ 1  0  0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
param_grid = {'C':[0.5,1,10,100,1000], 'gamma':[0.1,0.01,0.001,0.0001,0.00001]}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=True)

grid.fit(X_train,y_train)
grid_predictions = grid.predict(X_test)
print(classification_report(y_test,grid_predictions))
print(confusion_matrix(y_test,grid_predictions))

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


              precision    recall  f1-score   support

           0       0.78      0.86      0.82        21
           1       1.00      0.96      0.98        23
           2       0.37      0.85      0.52        20
           3       1.00      0.96      0.98        24
           4       0.89      1.00      0.94        16
           5       1.00      1.00      1.00        13
           6       0.38      0.14      0.20        22
           7       1.00      0.80      0.89        20
           8       0.86      0.90      0.88        20
           9       0.90      0.43      0.58        21

    accuracy                           0.78       200
   macro avg       0.82      0.79      0.78       200
weighted avg       0.81      0.78      0.77       200

[[18  0  0  0  0  0  0  0  3  0]
 [ 1 22  0  0  0  0  0  0  0  0]
 [ 0  0 17  0  0  0  3  0  0  0]
 [ 0  0  0 23  0  0  1  0  0  0]
 [ 0  0  0  0 16  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0 18  0  0  0  3  0  0  1]
 [ 2  0  0

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    2.9s finished


In [ ]:


m = RFECV(RandomForestClassifier(), scoring='accuracy')
m.fit(X_train, y_train)
m_predictions = m.predict(X_test)
print(classification_report(y_test,m_predictions))
print(confusion_matrix(y_test,m_predictions))
#Nakon PCA zavrsava izuzetno brze, manji accuracy doduse

              precision    recall  f1-score   support

           0       0.86      0.79      0.83        24
           1       0.84      0.88      0.86        24
           2       0.63      0.71      0.67        17
           3       1.00      0.95      0.97        19
           4       1.00      1.00      1.00        22
           5       1.00      0.95      0.97        19
           6       0.58      0.73      0.65        15
           7       0.86      0.86      0.86        14
           8       0.84      0.88      0.86        24
           9       0.72      0.59      0.65        22

    accuracy                           0.83       200
   macro avg       0.83      0.83      0.83       200
weighted avg       0.84      0.83      0.84       200

[[19  2  0  0  0  0  0  1  2  0]
 [ 0 21  0  0  0  0  0  1  2  0]
 [ 0  0 12  0  0  0  4  0  0  1]
 [ 0  0  1 18  0  0  0  0  0  0]
 [ 0  0  0  0 22  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  1]
 [ 0  0  1  0  0  0 11  0  0  3]
 [ 1  1  0

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_test)
print(classification_report(y_test,knn_predictions))
print(confusion_matrix(y_test,knn_predictions))

              precision    recall  f1-score   support

           0       0.72      0.86      0.78        21
           1       0.95      0.87      0.91        23
           2       0.45      0.65      0.53        20
           3       1.00      0.96      0.98        24
           4       0.94      1.00      0.97        16
           5       1.00      1.00      1.00        13
           6       0.53      0.36      0.43        22
           7       0.94      0.85      0.89        20
           8       0.89      0.85      0.87        20
           9       0.65      0.62      0.63        21

    accuracy                           0.79       200
   macro avg       0.81      0.80      0.80       200
weighted avg       0.80      0.79      0.79       200

[[18  1  0  0  0  0  0  0  2  0]
 [ 2 20  0  0  0  0  0  1  0  0]
 [ 0  0 13  0  0  0  4  0  0  3]
 [ 0  0  0 23  0  0  0  0  0  1]
 [ 0  0  0  0 16  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0 11  0  0  0  8  0  0  3]
 [ 2  0  0

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr_predictions = lr.predict(X_test)
print(classification_report(y_test,lr_predictions))
print(confusion_matrix(y_test,lr_predictions))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67        21
           1       0.88      0.91      0.89        23
           2       0.39      0.95      0.55        20
           3       1.00      0.96      0.98        24
           4       0.84      1.00      0.91        16
           5       1.00      1.00      1.00        13
           6       0.00      0.00      0.00        22
           7       0.94      0.85      0.89        20
           8       0.81      0.85      0.83        20
           9       0.58      0.33      0.42        21

    accuracy                           0.73       200
   macro avg       0.71      0.75      0.72       200
weighted avg       0.70      0.73      0.70       200

[[14  0  0  0  2  0  0  1  4  0]
 [ 2 21  0  0  0  0  0  0  0  0]
 [ 0  0 19  0  0  0  0  0  0  1]
 [ 0  0  1 23  0  0  0  0  0  0]
 [ 0  0  0  0 16  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  4]
 [ 2  0  0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)
dtc_predictions = dtc.predict(X_test)
print(classification_report(y_test,dtc_predictions))
print(confusion_matrix(y_test,dtc_predictions))

              precision    recall  f1-score   support

           0       0.68      0.71      0.70        21
           1       0.88      0.96      0.92        23
           2       0.54      0.65      0.59        20
           3       0.96      0.96      0.96        24
           4       1.00      0.94      0.97        16
           5       0.87      1.00      0.93        13
           6       0.68      0.59      0.63        22
           7       0.90      0.90      0.90        20
           8       0.76      0.65      0.70        20
           9       0.58      0.52      0.55        21

    accuracy                           0.78       200
   macro avg       0.79      0.79      0.78       200
weighted avg       0.78      0.78      0.78       200

[[15  1  0  0  0  1  0  1  3  0]
 [ 0 22  0  0  0  0  0  1  0  0]
 [ 0  0 13  0  0  0  3  0  0  4]
 [ 0  0  0 23  0  0  1  0  0  0]
 [ 1  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0  5  0  0  0 13  0  0  4]
 [ 1  0  0

Encoding and preprocessing test dataset


In [49]:
from sklearn.preprocessing import LabelEncoder
encodings_dictionary = dict()
for c in df.columns:
    if df[c].dtype == "object":
        encodings_dictionary[c] = LabelEncoder()
        df1[c] = encodings_dictionary[c].fit_transform(df1[c])

In [53]:
X = df1.drop('device_category',axis=1)
y= df1['device_category']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [57]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
print(classification_report(y_test,rf_predictions))
print(confusion_matrix(y_test,rf_predictions))


# 92% on the test set

                 precision    recall  f1-score   support

             TV       1.00      0.89      0.94        19
   baby_monitor       1.00      1.00      1.00        30
         lights       0.73      0.70      0.71        23
  motion_sensor       1.00      1.00      1.00        14
security_camera       0.91      1.00      0.95        21
 smoke_detector       1.00      1.00      1.00        22
         socket       0.53      0.57      0.55        14
     thermostat       1.00      1.00      1.00        17
          watch       1.00      1.00      1.00        20

       accuracy                           0.92       180
      macro avg       0.91      0.91      0.91       180
   weighted avg       0.92      0.92      0.92       180

[[17  0  0  0  2  0  0  0  0]
 [ 0 30  0  0  0  0  0  0  0]
 [ 0  0 16  0  0  0  7  0  0]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0 21  0  0  0  0]
 [ 0  0  0  0  0 22  0  0  0]
 [ 0  0  6  0  0  0  8  0  0]
 [ 0  0  0  0  0  0  0 17  0]
 [ 0  0  0  0  0